# EDA


- The os module has a perfect method to list files in a directory.
- Pandas json normalize could work here but is not necessary to convert the JSON data to a dataframe.
- You may need a nested for-loop to access each sale!
- We've put a lot of time into creating the structure of this repository, and it's a good example for future projects.  In the file functions_variables.py, there is an example function that you can import and use.  If you have any variables, functions or classes that you want to make, they can be put in the functions_variables.py file and imported into a notebook.  Note that only .py files can be imported into a notebook. If you want to import everything from a .py file, you can use the following:
```python
from functions_variables import *
```
If you just import functions_variables, then each object from the file will need to be prepended with "functions_variables"\
Using this .py file will keep your notebooks very organized and make it easier to reuse code between notebooks.

In [1]:
# (this is not an exhaustive list of libraries)
import pandas as pd
import numpy as np
import os
import json
from pprint import pprint
from functions_variables import encode_tags

## Data Importing

In [2]:
# load one file first to see wha# Folder containing all the JSON files
json_folder = '/home/t0si/LHL-Midterm-Project/data/housing'

# List to hold individual DataFrames
data_frames = []



  

In [3]:
# loop over all files and put them into a dataframe# Iterate through each file in the folder
for file in os.listdir(json_folder):
    if file.endswith('.json'):  # Ensure it's a JSON file
        file_path = os.path.join(json_folder, file)
        print(f"Processing file: {file}")
        with open(file_path, 'r') as f:
            data = json.load(f)
            results = data.get('data', {}).get('results', [])
            
            if not results:
                print(f"No valid data in {file}. Skipping...")
                continue
            
            try:
                # Flatten the JSON structure for each file
                df = pd.json_normalize(
                    results,
                    sep='_',
                    meta=[
                        'list_date',
                        'list_price',
                        'status',
                        ['description', 'year_built'],
                        ['description', 'sqft'],
                        ['description', 'beds'],
                        ['description', 'baths'],
                        ['location', 'address', 'city'],
                        ['location', 'address', 'state'],
                        ['location', 'address', 'postal_code'],
                    ],
                    errors='ignore'
                )
                # Drop any columns that are all NaN
                df.dropna(axis=1, how='all', inplace=True)
                print(f"File {file} processed. Rows: {len(df)}")
                data_frames.append(df)  # Add the DataFrame to the list
            except Exception as e:
                print(f"Error processing {file}: {e}")

Processing file: IL_Springfield_3.json
File IL_Springfield_3.json processed. Rows: 34
Processing file: SC_Columbia_3.json
File SC_Columbia_3.json processed. Rows: 31
Processing file: WY_Cheyenne_2.json
No valid data in WY_Cheyenne_2.json. Skipping...
Processing file: KY_Frankfort_2.json
File KY_Frankfort_2.json processed. Rows: 42
Processing file: WY_Cheyenne_3.json
No valid data in WY_Cheyenne_3.json. Skipping...
Processing file: VT_Montpelier_2.json
No valid data in VT_Montpelier_2.json. Skipping...
Processing file: NJ_Trenton_4.json
File NJ_Trenton_4.json processed. Rows: 42
Processing file: VA_Richmond_4.json
File VA_Richmond_4.json processed. Rows: 42
Processing file: OK_OklahomaCity_1.json
File OK_OklahomaCity_1.json processed. Rows: 42
Processing file: FL_Tallahassee_2.json
File FL_Tallahassee_2.json processed. Rows: 42
Processing file: WA_Olympia_4.json
File WA_Olympia_4.json processed. Rows: 42
Processing file: MN_St.Paul_2.json
File MN_St.Paul_2.json processed. Rows: 42
Proce

## Data Cleaning and Wrangling

At this point, ensure that you have all sales in a dataframe.
- Take a quick look at your data (i.e. `.info()`, `.describe()`) - what do you see?
- Is each cell one value, or do some cells have lists?
- What are the data types of each column?
- Some sales may not actually include the sale price (target).  These rows should be dropped.
- There are a lot of NA/None values.  Should these be dropped or replaced with something?
    - You can drop rows or use various methods to fills NA's - use your best judgement for each column 
    - i.e. for some columns (like Garage), NA probably just means no Garage, so 0
- Drop columns that aren't needed
    - Don't keep the list price because it will be too close to the sale price. Assume we want to predict the price of houses not yet listed

In [4]:
# load and concatenate data here
# drop or replace values as necessary# Combine all DataFrames into one
if data_frames:  
    combined_df = pd.concat(data_frames, ignore_index=True)
else:
    combined_df = pd.DataFrame()

In [5]:
columns = combined_df.columns.sort_values().to_list()
combined_df = combined_df[columns]

In [6]:
# drop or replace values as necessary
combined_df = combined_df.drop(
    columns=[
        'branding',
        'community_advertisers',
        'community_description_name',
        # 'description.baths',
        'description_baths_3qtr',
        'description_baths_full',
        'description_baths_half',
        # 'description.beds',
        # 'description.garage',
        # 'description.lot_sqft',
        # 'description.sold_date',
        # 'description.sold_price',
        # 'description.sqft',
        # 'description.stories',
        'description_sub_type',
        'description_type',
        # 'description.year_built',
         'flags_is_foreclosure',
         'flags_is_new_listing',
         'flags_is_price_reduced',
        'last_update_date',
        'lead_attributes_show_contact_an_agent',
        # 'list_date',
        # 'list_price',
        'listing_id',
        # 'location.address.city',
         'location_address_coordinate_lat',
         'location_address_coordinate_lon',
        'location_address_line',
        # 'location.address.postal_code',
        # 'location.address.state',
         'location_address_state_code',
        # 'location.county.fips_code',
        # 'location.county.name',
        'location_street_view_url',
        'matterport',
        'other_listings_rdc',
        'permalink',
        'photos',
        # 'price_reduced_amount',
        'primary_photo_href',
        '#products_brand_name',
        # 'property_id',
        'source_agents',
        'source_plan_id',
        'source_spec_id',
        'source_type',
        'status',
        # 'tags',
        'virtual_tours'
    ],
    errors='ignore'
)

In [7]:
#drop all the rows that dont have a listing price
combined_df = combined_df.dropna(subset=['list_price'])

In [8]:
#add 0 to all the rows that have description_garage as NaN
combined_df['description_garage'] = combined_df['description_garage'].fillna(0)

In [11]:
#check and count the different values in the column description_stories
combined_df['description_stories'].value_counts()


description_stories
1.0    3224
2.0    2449
3.0     432
4.0      25
6.0      16
Name: count, dtype: int64

In [26]:
#count values on the column description_sold_price and put them in ascending order set option to display all the rows
#combined_df['description_sold_price'].value_counts().sort_index(ascending=True)

pd.set_option('display.max_rows', None)  # Show all rows
print(combined_df['description_sold_price'].value_counts().sort_index(ascending=True))

#check for outliers in the column description_sold_price
combined_df['description_sold_price'].describe()




description_sold_price
308.0          5
385.0          5
583.0          5
1122.0        10
2500.0         5
2585.0         5
3575.0         5
3800.0         5
4500.0         5
5000.0         5
5900.0         5
6380.0         5
6930.0         5
7590.0         5
7700.0         5
9000.0         5
9900.0         5
10000.0        5
12900.0        3
13200.0        2
14300.0        5
14900.0        5
15400.0       10
15900.0        5
16500.0        5
18900.0        5
19900.0        5
20700.0        5
20900.0        5
22000.0       10
25000.0       13
26000.0        5
27225.0        5
27500.0        5
28000.0       15
28600.0        5
29700.0       10
29900.0        5
30000.0       10
31500.0        5
32500.0        1
32900.0        6
35000.0        5
38500.0        3
39900.0        5
40000.0        5
41800.0        5
42000.0        5
42900.0       10
45000.0        5
48500.0        5
49900.0        5
50000.0        9
51860.0        5
53325.0        5
53591.0        5
54167.0        5
55000.0 

count    6.422000e+03
mean     3.964653e+05
std      5.215674e+05
min      3.080000e+02
25%      1.900000e+05
50%      3.100000e+05
75%      4.600000e+05
max      1.250000e+07
Name: description_sold_price, dtype: float64

In [ ]:
#check for outliers in the column list_price

Series([], Name: count, dtype: int64)
description_beds
3.0     586
4.0     266
2.0     234
5.0      69
1.0      39
6.0      21
8.0       5
9.0       5
12.0      5
Name: count, dtype: int64


In [10]:
#drop all the rows that have description_stories = 10.0, 8.0
combined_df = combined_df[~combined_df['description_stories'].isin([10.0, 8.0])]


In [27]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7714 entries, 1073 to 8074
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   description_baths             7569 non-null   float64
 1   description_beds              7376 non-null   float64
 2   description_garage            7714 non-null   float64
 3   description_lot_sqft          6839 non-null   float64
 4   description_sold_date         7714 non-null   object 
 5   description_sold_price        6422 non-null   float64
 6   description_sqft              7214 non-null   float64
 7   description_stories           6146 non-null   float64
 8   description_year_built        7199 non-null   float64
 9   list_date                     7714 non-null   object 
 10  list_price                    7714 non-null   float64
 11  location_address_city         7709 non-null   object 
 12  location_address_postal_code  7714 non-null   object 
 13  locat

### Dealing with Tags

Consider the fact that with tags, there are a lot of categorical variables.
- How many columns would we have if we OHE tags, city and state?
- Perhaps we can get rid of tags that have a low frequency.

In [ ]:
# Save the combined DataFrame to a CSV file
output_file = 'combined_cleaned_data.csv'
combined_df.to_csv(output_file, index=False)

In [ ]:
combined_df.describe()

In [ ]:
# OHE categorical variables/ tags here
# tags will have to be done manually

### Dealing with Cities

- Sales will vary drastically between cities and states.  Is there a way to keep information about which city it is without OHE?
- Could we label encode or ordinal encode?  Yes, but this may have undesirable effects, giving nominal data ordinal values.
- What we can do is use our training data to encode the mean sale price by city as a feature (a.k.a. Target Encoding)
    - We can do this as long as we ONLY use the training data - we're using the available data to give us a 'starting guess' of the price for each city, without needing to encode city explicitly
- If you replace cities or states with numerical values (like the mean price), make sure that the data is split so that we don't leak data into the training selection. This is a great time to train test split. Compute on the training data, and join these values to the test data
- Note that you *may* have cities in the test set that are not in the training set. You don't want these to be NA, so maybe you can fill them with the overall mean

In [ ]:
# perform train test split here

# do something with state and city

## Extra Data - STRETCH

> This doesn't need to be part of your Minimum Viable Product (MVP). We recommend you write a functional, basic pipeline first, then circle back and join new data if you have time

> If you do this, try to write your downstream steps in a way it will still work on a dataframe with different features!

- You're not limited to just using the data provided to you. Think/ do some research about other features that might be useful to predict housing prices. 
- Can you import and join this data? Make sure you do any necessary preprocessing and make sure it is joined correctly.
- Example suggestion: could mortgage interest rates in the year of the listing affect the price? 

In [ ]:
# import, join and preprocess new data here

## EDA/ Visualization

Remember all of the EDA that you've been learning about?  Now is a perfect time for it!
- Look at distributions of numerical variables to see the shape of the data and detect outliers.    
    - Consider transforming very skewed variables
- Scatterplots of a numerical variable and the target go a long way to show correlations.
- A heatmap will help detect highly correlated features, and we don't want these.
    - You may have too many features to do this, in which case you can simply compute the most correlated feature-pairs and list them
- Is there any overlap in any of the features? (redundant information, like number of this or that room...)

In [ ]:
# perform EDA here

## Scaling and Finishing Up

Now is a great time to scale the data and save it once it's preprocessed.
- You can save it in your data folder, but you may want to make a new `processed/` subfolder to keep it organized